In [1]:
#initialize model
from loss import ContrastiveLoss
from model import *
import torch
from ml_training import *
from dl_training import *


# Resnet 50

In [3]:
criterion=ContrastiveLoss()
res50_model=ResNet50()
opt=torch.optim.Adam(params=res50_model.parameters(),lr=0.0001)

In [4]:
#train model
res50_model_trained=train_dl(50,res50_model,"cuda",criterion,opt)

In [5]:
#extract features
fracture=get_features('../Wrist Fracture/train/Fracture',res50_model_trained)
normal= get_features('../Wrist Fracture/train/Normal',res50_model_trained)

In [6]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [7]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [8]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.006981,0.085970,0.019947,0.015752
score_time,0.007180,0.008777,0.006981,0.015344
test_accuracy,0.784492,0.791087,0.760963,0.797504
train_accuracy,0.878139,0.842003,0.793791,1.000000
test_precision_macro,0.807667,0.824355,0.789343,0.818295
train_precision_macro,0.874748,0.841925,0.788775,1.000000
test_recall_macro,0.783872,0.799323,0.759850,0.814511
train_recall_macro,0.879170,0.849880,0.788337,1.000000
test_f1_macro,0.776199,0.780644,0.748183,0.784225
train_f1_macro,0.876051,0.840991,0.788471,1.000000


In [9]:
#embeddings of test data
test_fracture=get_features('../Wrist Fracture/test/Fracture',res50_model_trained)
test_normal= get_features('../Wrist Fracture/test/Normal',res50_model_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [10]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [11]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.75      0.50      0.60        12
           1       0.68      0.87      0.76        15

    accuracy                           0.70        27
   macro avg       0.72      0.68      0.68        27
weighted avg       0.71      0.70      0.69        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.64      0.75      0.69        12
           1       0.77      0.67      0.71        15

    accuracy                           0.70        27
   macro avg       0.71      0.71      0.70        27
weighted avg       0.71      0.70      0.70        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.78      0.58      0.67        12
           1       0.72      0.87      0.79        15

    accuracy                           0.74        

# My Model V1

In [12]:
criterion=ContrastiveLoss()
v2_model=SqueezeNetV2()
opt=torch.optim.Adam(params=v2_model.parameters(),lr=0.0001)

In [13]:
#train model
v2_model_trained=train_dl(50,v2_model,"cuda",criterion,opt)

In [14]:
#extract features
fracture=get_features('../Wrist Fracture/train/Fracture',v2_model_trained)
normal= get_features('../Wrist Fracture/train/Normal',v2_model_trained)

In [15]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [16]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [17]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_

,dt,svm,lr,knn
fit_time,0.008187,0.071409,0.065218,0.013962
score_time,0.007379,0.007780,0.006982,0.013974
test_accuracy,0.809091,0.803030,0.766845,0.809091
train_accuracy,0.826920,0.813374,0.817897,0.801321
test_precision_macro,0.819894,0.823853,0.797890,0.824880
train_precision_macro,0.831489,0.813834,0.813701,0.808300
test_recall_macro,0.818346,0.809586,0.763233,0.822368
train_recall_macro,0.836423,0.820318,0.813534,0.813303
test_f1_macro,0.805063,0.795467,0.751892,0.803514
train_f1_macro,0.826018,0.812102,0.813324,0.800738


In [18]:
#embeddings of test data
test_fracture=get_features('../Wrist Fracture/test/Fracture',v2_model_trained)
test_normal= get_features('../Wrist Fracture/test/Normal',v2_model_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [19]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [20]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.71      0.83      0.77        12
           1       0.85      0.73      0.79        15

    accuracy                           0.78        27
   macro avg       0.78      0.78      0.78        27
weighted avg       0.79      0.78      0.78        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.71      0.83      0.77        12
           1       0.85      0.73      0.79        15

    accuracy                           0.78        27
   macro avg       0.78      0.78      0.78        27
weighted avg       0.79      0.78      0.78        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.70      0.58      0.64        12
           1       0.71      0.80      0.75        15

    accuracy                           0.70        

C:\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# My Model V2

In [21]:
criterion=ContrastiveLoss()
v3_model=SqueezeNetV3()
opt=torch.optim.Adam(params=v3_model.parameters(),lr=0.0001)

In [22]:
#train model
v3_model_trained=train_dl(50,v3_model,"cuda",criterion,opt)

In [23]:
#extract features
fracture=get_features('../Wrist Fracture/train/Fracture',v3_model_trained)
normal= get_features('../Wrist Fracture/train/Normal',v3_model_trained)

In [24]:
#features and labels
feature=np.array(fracture+normal)
label=np.array([0]*len(normal)+[1]*len(fracture))

In [25]:
#tune ml hyper parameters
dt_clf,_=classifiers_tuning('dt',feature,label)
svm_clf,_=classifiers_tuning('svm',feature,label)
knn_clf,_=classifiers_tuning('knn',feature,label)
lr_clf,_=classifiers_tuning('lr',feature,label)

In [26]:
#display 5 fold cross validation result
dt_result=display_result(dt_clf,feature,label)
svm_result=display_result(svm_clf,feature,label)
knn_result=display_result(knn_clf,feature,label)
lr_result=display_result(lr_clf,feature,label)
df=pd.DataFrame(zip(dt_result,svm_result,lr_result,knn_result),index=dt_result.index,columns=['dt','svm','lr','knn'])
df


,dt,svm,lr,knn
fit_time,0.006582,0.071022,0.022546,0.014222
score_time,0.006383,0.006971,0.006570,0.016134
test_accuracy,0.814795,0.802852,0.784670,0.808913
train_accuracy,0.829950,0.831431,0.825439,0.783219
test_precision_macro,0.846705,0.858818,0.846399,0.854853
train_precision_macro,0.837440,0.828754,0.821103,0.785741
test_recall_macro,0.829248,0.806090,0.782782,0.816992
train_recall_macro,0.838585,0.833988,0.821505,0.791353
test_f1_macro,0.809248,0.787788,0.766359,0.796299
train_f1_macro,0.828679,0.829289,0.821245,0.781974


In [27]:
#embeddings of test data
test_fracture=get_features('../Wrist Fracture/test/Fracture',v3_model_trained)
test_normal= get_features('../Wrist Fracture/test/Normal',v3_model_trained)
test_feature=np.array(test_fracture+test_normal)
test_label=np.array([0]*len(test_normal)+[1]*len(test_fracture))

In [28]:
#scale the data
scaler=StandardScaler()
feature=scaler.fit_transform(feature)
test_feature=scaler.transform(test_feature)

In [29]:
for clf_model,clf_name in zip([dt_clf,svm_clf,lr_clf,knn_clf],\
                              ['Decision Tree','SVM',"logistic regression",'K Nearest neighbors']):
    print('-----------------{}--------------'.format(clf_name))
    clf_model.fit(feature,label)
    test_pred=clf_model.predict(test_feature)
    print(classification_report(test_label,test_pred))

-----------------Decision Tree--------------
              precision    recall  f1-score   support

           0       0.77      0.83      0.80        12
           1       0.86      0.80      0.83        15

    accuracy                           0.81        27
   macro avg       0.81      0.82      0.81        27
weighted avg       0.82      0.81      0.82        27

-----------------SVM--------------
              precision    recall  f1-score   support

           0       0.67      0.50      0.57        12
           1       0.67      0.80      0.73        15

    accuracy                           0.67        27
   macro avg       0.67      0.65      0.65        27
weighted avg       0.67      0.67      0.66        27

-----------------logistic regression--------------
              precision    recall  f1-score   support

           0       0.75      0.50      0.60        12
           1       0.68      0.87      0.76        15

    accuracy                           0.70        